In [1]:
!pip install python-dotenv
!pip install -U torch transformers
!pip install bitsandbytes
!pip install datasets
!apt install git-lfs
!pip install accelerate
!pip install nltk
!pip install evaluate
!pip install rouge_score

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [2]:
#Get acccess token from Hugging Face, and agree to conditions of use for Gemma

from huggingface_hub import notebook_login

notebook_login()

# Gemma Model

In [3]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

#Load the model, its corresponding tokenizer, and quantization config

model_id = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_use_double_quant=True,
                                         bnb_4bit_compute_dtype=torch.bfloat16)

#Use quantization config if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"using {device}")

if device == 'cuda':
  model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map="auto",
                                             quantization_config=quantization_config
                                             )
else:
  model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map="auto"
                                             )
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaR

In [4]:
#This function is to prepare the prompt and do inference using the Gemma model

def inference(question: str, context: str):

    if context == None or context == "":
        prompt = f"""Give a detailed answer to the following question. Question: {question}"""
    else:
        prompt = f"""Using the information contained in the context, give a detailed answer to the question.
            Context: {context}.
            Question: {question}"""
    chat = [
        {"role": "user", "content": prompt}
    ]
    formatted_prompt = tokenizer.apply_chat_template(
        chat,
        tokenize=False,
        add_generation_prompt=True,
    )
    inputs = tokenizer.encode(
        formatted_prompt, add_special_tokens=False, return_tensors="pt"
    ).to(device)
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=100,
            do_sample=False,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    response = response[len(formatted_prompt) :]  # remove input prompt from reponse
    response = response.replace("<eos>", "")  # remove eos token
    return response

In [ ]:
#Try the generation of Gemma with one question

question = "What is a good AI research idea?"
print(inference(question=question, context=""))

**A good AI research idea should meet the following criteria:**

**1. Relevance:**
* The research should address a significant problem or gap in the field.
* It should have potential real-world applications or impact.

**2. Feasibility:**
* The research question should be well-defined and have a clear path to achieving the desired outcome.
* The required data and resources should be accessible and cost-effective to acquire.

**3. Impact:**
* The research


# Evaluate the model on a RAG dataset

In [ ]:
#In order to see how evaluate the generation results of the Gemma model, I get a RAG dataset from HuggingFace

from datasets import load_dataset

datasets = load_dataset("neural-bridge/rag-dataset-12000")

Generating train split:   0%|          | 0/9600 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2400 [00:00<?, ? examples/s]

In [ ]:
#This dataset has context, question, and answer, and I can estimate how good Gemma's answer is compared to the "ground truth" answer

datasets

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 9600
    })
    test: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 2400
    })
})

In [ ]:
#This function is to view some rows of the dataset

from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

In [ ]:
#Taking an example from the dataset

question = datasets['train'][0]['question']
context = datasets['train'][0]['context']
answer = datasets['train'][0]['answer']

print(question +"\n\n"+context+"\n\n"+answer)

What is the Berry Export Summary 2028 and what is its purpose?

Caption: Tasmanian berry grower Nic Hansen showing Macau chef Antimo Merone around his property as part of export engagement activities.
THE RISE and rise of the Australian strawberry, raspberry and blackberry industries has seen the sectors redouble their international trade focus, with the release of a dedicated export plan to grow their global presence over the next 10 years.
Driven by significant grower input, the Berry Export Summary 2028 maps the sectors’ current position, where they want to be, high-opportunity markets and next steps.
Hort Innovation trade manager Jenny Van de Meeberg said the value and volume of raspberry and blackberry exports rose by 100 per cent between 2016 and 2017. She said the Australian strawberry industry experienced similar success with an almost 30 per cent rise in export volume and a 26 per cent rise in value to $32.6M over the same period.
“Australian berry sectors are in a firm positi

In [ ]:
#Do inference using the question and context

infer = inference(question=question, context=context)

print(infer)

The Berry Export Summary 2028 is a comprehensive map that outlines the current position and future prospects of the Australian strawberry, raspberry and blackberry industries. It is a valuable resource for growers, exporters and other stakeholders in the berry industry to understand the opportunities for growth and development in the global market.

The purpose of the Berry Export Summary 2028 is to provide a clear and concise overview of the industry's current and future landscape, including:

- High-income countries with


In [ ]:
#Importing nltk to calculate BLEU score

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Function to compute BLEU score

from nltk.translate.bleu_score import sentence_bleu

def compute_bleu(reference, hypothesis):
    # Tokenize the reference and hypothesis sentences
    ref_tokens = nltk.word_tokenize(reference.lower())
    hyp_tokens = nltk.word_tokenize(hypothesis.lower())

    # Compute BLEU score
    bleu_score = sentence_bleu([ref_tokens], hyp_tokens)

    return bleu_score

# Calculate BLEU score to compare the answer with the inference result
bleu_score = compute_bleu(answer, infer)
print(f"BLEU Score: {bleu_score:.4f}")

BLEU Score: 0.1939


In [ ]:
#It looks like BLEU score is 19%. We can see the answer and inference side by side below

print(answer)

print(infer)

The Berry Export Summary 2028 is a dedicated export plan for the Australian strawberry, raspberry, and blackberry industries. It maps the sectors’ current position, where they want to be, high-opportunity markets, and next steps. The purpose of this plan is to grow their global presence over the next 10 years.
The Berry Export Summary 2028 is a comprehensive map that outlines the current position and future prospects of the Australian strawberry, raspberry and blackberry industries. It is a valuable resource for growers, exporters and other stakeholders in the berry industry to understand the opportunities for growth and development in the global market.

The purpose of the Berry Export Summary 2028 is to provide a clear and concise overview of the industry's current and future landscape, including:

- High-income countries with


In [ ]:
#Here I get ROUGE to calculate another metric

import evaluate

rouge = evaluate.load('rouge')

In [ ]:
#Tokenize the sentences in the right way for ROUGE score

prediction = nltk.sent_tokenize(infer)
reference = nltk.sent_tokenize(answer)
print(prediction, reference)

['The Berry Export Summary 2028 is a comprehensive map that outlines the current position and future prospects of the Australian strawberry, raspberry and blackberry industries.', 'It is a valuable resource for growers, exporters and other stakeholders in the berry industry to understand the opportunities for growth and development in the global market.', "The purpose of the Berry Export Summary 2028 is to provide a clear and concise overview of the industry's current and future landscape, including:\n\n- High-income countries with"] ['The Berry Export Summary 2028 is a dedicated export plan for the Australian strawberry, raspberry, and blackberry industries.', 'It maps the sectors’ current position, where they want to be, high-opportunity markets, and next steps.', 'The purpose of this plan is to grow their global presence over the next 10 years.']


In [ ]:
#Get the ROUGE score, which looks like 37% if we look at ROUGE-1

results = rouge.compute(predictions=prediction,
                       references=reference)

print(results)

{'rouge1': 0.366549213060841, 'rouge2': 0.24163357912648895, 'rougeL': 0.366549213060841, 'rougeLsum': 0.366549213060841}


# Conclusions

BASED ON BLEU score of 19% and ROUGE score of 37%, it looks like the inference is similar to the original answer to an extent.

However, it is also clear that this metric is not very helpful in this case, because if we evaluate the results as a human, the inference is actually informative and just as helpful as the original answer, even though the words used are not the same.

We will need to come up with better ways to evaluate generated text in the future.

As next step, we can also calculate BLEU and ROUGE for the whole RAG dataset to get an average number. We can also try finetuning Gemma on this dataset, and see if it performs better after training.

# Finetuning

In this section, I try use LORA and Supervised Fine Tunning (SFFT) to finetune the Gemma model with the RAG dataset.

I formatted the prompt to be appropriate for training, and used a LORA config, however, I haven't been able to succesfully run the trainer. The error is CUDA out of memory.

My next step to be to adjust the LORA Config and also the SFFT config, as well as the size of the training dataset so training can happen.

In [ ]:
!pip install -q -U peft
!pip install -q -U trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00


In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [ ]:
#Prompt formatting for training

def generate_prompt(data_point):
    prefix_text = 'Below is a context for the question. Give an answer that ' \
               'appropriately answers the question.\n\n'
    # Samples with additional context into.
    if data_point['context']:
        text = f"""<start_of_turn>user {prefix_text} {data_point["context"]} here is the question {data_point["question"]} <end_of_turn>\n<start_of_turn>model{data_point["answer"]} <end_of_turn>"""
    # Without
    else:
        text = f"""<start_of_turn>user {prefix_text} {data_point["question"]} <end_of_turn>\n<start_of_turn>model{data_point["answer"]} <end_of_turn>"""
    return text

# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in datasets['train']]
datasets['train'] = datasets['train'].add_column("prompt", text_column)

In [ ]:
#Adjust the dataset to include a prompt and text column

datasets['train'] = datasets['train'].map(lambda samples: tokenizer(samples["prompt"]), batched=True)
datasets['train'] = datasets['train'].add_column("text", text_column)
datasets['train'] = datasets['train'].map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/9600 [00:00<?, ? examples/s]

In [ ]:
dataset = datasets["train"]

print(dataset)

Dataset({
    features: ['context', 'question', 'answer', 'prompt', 'input_ids', 'attention_mask', 'text'],
    num_rows: 9600
})


In [ ]:
#Split the dataset to smaller train and test datasets

dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import transformers

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
#The training function

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    peft_config=lora_config)

trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


OutOfMemoryError: CUDA out of memory. Tried to allocate 42.00 MiB. GPU 

In [ ]:
#Saving model

new_model = "gemma-Finetune"
trainer.model.save_pretrained(new_model)

In [ ]:
# Merge the model with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

In [ ]:
#Try the new model on the same question and context

question = datasets['train'][0]['question']
context = datasets['train'][0]['context']

model = merged_model

infer = inference(question=question, context=context)

print(infer)

# Document Loading and Splitting

This section is for loading and tokenizing the document using Langchain.

In [5]:
!pip install pypdf
!pip install tiktoken
!pip install -qU langchain
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.9 MB/s eta 0:00:00


In [6]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
#Create folder data and put the pdf document in that folder

from langchain.document_loaders import PyPDFLoader

loaders = [
    PyPDFLoader("/content/drive/My Drive/data/ResearchProposal.pdf")
]
pages = []
for loader in loaders:
    pages.extend(loader.load())

In [8]:
#Split text after loading document

from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=12)
docs = text_splitter.split_documents(pages)

In [9]:
print(docs[0].page_content)

Research
proposal
Master
of
Research
-
Business
Analytics
-
S2
2024
Prepared
by
Taylor
Chu
-
LinkedIn
-
hathanh.chu@students.mq.edu.au
AREA
OF
RESEARCH
-
RESPONSIBLE
AI
Since
the
beginning
of
the
internet
and
especially
over
the
recent
pandemic,
we
have
evolved
to
live
much
of
our
lives
online,
and


# Embeddings and Vector Store

In [10]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 2.1 MB/s eta 0:00:00


In [11]:
#Import HuggingFace Embeddings using sentence transformers

import numpy as np
from langchain_community.embeddings import (
    HuggingFaceEmbeddings
)
encoder = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2', model_kwargs = {'device': "cpu"})


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
#Example of how embeddings result look, and taking dot product of two embeddings

embeddings1 = encoder.embed_query("Document intelligence")
embeddings2 = encoder.embed_query(docs[0].page_content)
print(np.dot(embeddings1, embeddings2))

0.35274698992279585


In [13]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 25.7 MB/s eta 0:00:00


In [14]:
#Using FAISS with distance strategy Dot product to calculate similarity between query and document context pieces

from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
faiss_db = FAISS.from_documents(docs, encoder, distance_strategy=DistanceStrategy.DOT_PRODUCT)

In [15]:
#Asked question find the 5 most similar context pieces (defined by k = 5)

question = "What is the research topic?"
retrieved_docs = faiss_db.similarity_search(question, k=5)
context = "".join(doc.page_content + "\n" for doc in retrieved_docs)
print(context)

and
for
topic
modeling,
I
will
use
a
State
of
the
Art
(SOTA)
model,
the
BERTopic
model.
As
I
read
the
literature,
I
will
identify
areas
of
potential
tension,
conﬂict
or
alignment
between
government,
industry
and
research,
and
attempt
to
verify
these
themes
through
analyzing
the
Australian
government’s
public

text
more
quickly,
and
ﬁnd
patterns
that
may
be
missed
by
a
human
reader
and
analyst.
As
this
area
of
research
continues
to
grow
and
expand,
the
method
can
be
utilized
to
help
the
government
more
quickly
assess
the
submissions
they
receive
from
public
consultations,
or
also
help
other
industry
and
research
institutions

as
the
510
submissions
from
the
public.
I
will
use
techniques
like
topic
modeling
and
sentiment
analysis
to
extract
the
areas
of
focus
in
each
submission.
As
the
submissions
have
diﬀerent
formatting
and
structure,
but
still
mostly
attempt
to
address
the
20
questions
put
forth
by
the
government’s
discussion
paper,
I
will
�s
discussion
paper,
I
will
need
to
extract
t

In [16]:
torch.cuda.empty_cache()

#Do inference using Gemma on the question and chosen pieces of context. The answer is pretty good in this case.

print(inference(question=question, context=context))

The research topic is the analysis of the Australian government's Safe and Responsible AI discussion paper and the 510 submissions from the public.


In [17]:
#Quote where the info was taken from

print("For this answer I used the following documents:")
for doc in retrieved_docs:
    print(doc.metadata)

For this answer I used the following documents:
{'source': '/content/drive/My Drive/data/ResearchProposal.pdf', 'page': 2}
{'source': '/content/drive/My Drive/data/ResearchProposal.pdf', 'page': 3}
{'source': '/content/drive/My Drive/data/ResearchProposal.pdf', 'page': 2}
{'source': '/content/drive/My Drive/data/ResearchProposal.pdf', 'page': 2}
{'source': '/content/drive/My Drive/data/ResearchProposal.pdf', 'page': 2}
